In [1]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
import dask.config
from distributed import LocalCluster, Client

In [2]:
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=4,
                       threads_per_worker=2,
                       memory_limit='3 GB')
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33883 Dashboard: http://127.0.0.1:8790/status,Cluster Workers: 4 Cores: 8 Memory: 12.00 GB


In [3]:
df = dd.read_parquet('../data/contiguous_us_w_geohash.parquet', engine='pyarrow')
display(df.head(2))
len(df)

,latitude,longitude,geohash
index,,,
38605,24.396309,-75.881459,dk64c30ycce4
38630,24.396317,-109.907775,9s3ffcpwg5p1


113944489

In [4]:
%%time 
t0 = time.time()
df['geohash'] = df['geohash'].str[:4]
df.set_index('geohash', shuffle='disk').to_parquet('../data/contiguous_us_geohash4_sorted.parquet', engine='pyarrow')
dt_hr = (time.time() - t0)/60/60

CPU times: user 19 s, sys: 2.1 s, total: 21.1 s
Wall time: 2min 43s


In [5]:
with open(f'us_geohash_sort_time_{datetime.now()}.csv', 'w') as f:
    f.write(f'dt_hr, {dt_hr}')

In [6]:
df = dd.read_parquet('../data/contiguous_us_geohash4_sorted.parquet', engine='pyarrow')
display(df.head(2))
len(df)

,latitude,longitude
geohash,,
9hre,24.591,-124.248
9hre,24.447,-124.443


113944489